In [8]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim

In [16]:
# 데이터 경로 설정
base_dir = "C:/Users/Admin/Desktop/모션저장"
categories = ["등", "스쿼트", "옆구리"]

In [17]:
def load_data(base_dir, categories, seq_len=30):
    """
    데이터 로드 및 크기 조정 (패딩 또는 잘라내기).
    Args:
        base_dir (str): 데이터가 저장된 기본 경로.
        categories (list): 행동 카테고리 리스트.
        seq_len (int): 고정된 시퀀스 길이.
    Returns:
        np.array: 데이터 배열.
        np.array: 라벨 배열.
    """
    data = []
    labels = []
    label_map = {category: idx for idx, category in enumerate(categories)}  # 라벨 매핑

    for category in categories:
        category_path = os.path.join(base_dir, category, "좌표")
        print(f"Checking files in: {category_path}")
        for i in range(1, 11):  # keypoints_1 ~ keypoints_10
            file_path = os.path.join(category_path, f"keypoints_{i}.npy")
            if os.path.exists(file_path):
                keypoints = np.load(file_path)  # 데이터 로드
                if keypoints.shape[0] > seq_len:  # 길이가 seq_len보다 길면 잘라내기
                    keypoints = keypoints[:seq_len]
                elif keypoints.shape[0] < seq_len:  # 길이가 seq_len보다 짧으면 패딩
                    pad_width = seq_len - keypoints.shape[0]
                    keypoints = np.pad(keypoints, ((0, pad_width), (0, 0), (0, 0)), mode='constant')
                data.append(keypoints)
                labels.append(label_map[category])
            else:
                print(f"File not found: {file_path}")

    return np.array(data), np.array(labels)

# 수정된 데이터 로드 실행
data, labels = load_data(base_dir, categories)
print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")

Checking files in: C:/Users/Admin/Desktop/모션저장\등\좌표
Checking files in: C:/Users/Admin/Desktop/모션저장\스쿼트\좌표
Checking files in: C:/Users/Admin/Desktop/모션저장\옆구리\좌표
Data shape: (30, 30, 33, 3), Labels shape: (30,)


In [18]:
# 데이터 로드
data, labels = load_data(base_dir, categories)
print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")

Checking files in: C:/Users/Admin/Desktop/모션저장\등\좌표
Checking files in: C:/Users/Admin/Desktop/모션저장\스쿼트\좌표
Checking files in: C:/Users/Admin/Desktop/모션저장\옆구리\좌표
Data shape: (30, 30, 33, 3), Labels shape: (30,)


In [20]:
import os
import numpy as np

# 데이터 경로 및 라벨 매핑
data_paths = {
    "등": "C:/Users/Admin/Desktop/모션저장/등/좌표",
    "스쿼트": "C:/Users/Admin/Desktop/모션저장/스쿼트/좌표",
    "옆구리": "C:/Users/Admin/Desktop/모션저장/옆구리/좌표"
}
labels_map = {"등": 0, "스쿼트": 1, "옆구리": 2}  # 라벨 매핑

def load_data(data_paths, labels_map, seq_len=30):
    """
    데이터 로드 및 크기 조정 (패딩 또는 잘라내기).
    Args:
        data_paths (dict): 카테고리별 데이터 경로.
        labels_map (dict): 카테고리와 라벨 매핑.
        seq_len (int): 고정된 시퀀스 길이.
    Returns:
        np.array: 데이터 배열.
        np.array: 라벨 배열.
    """
    data = []
    labels = []
    for category, path in data_paths.items():
        print(f"Checking files in: {path}")
        for i in range(1, 11):  # keypoints_1 ~ keypoints_10
            file_path = os.path.join(path, f"keypoints_{i}.npy")
            if os.path.exists(file_path):
                keypoints = np.load(file_path)  # 데이터 로드
                if keypoints.shape[0] > seq_len:  # 길이가 seq_len보다 길면 잘라내기
                    keypoints = keypoints[:seq_len]
                elif keypoints.shape[0] < seq_len:  # 길이가 seq_len보다 짧으면 패딩
                    pad_width = seq_len - keypoints.shape[0]
                    keypoints = np.pad(keypoints, ((0, pad_width), (0, 0), (0, 0)), mode='constant')
                data.append(keypoints)
                labels.append(labels_map[category])
            else:
                print(f"File not found: {file_path}")
    return np.array(data), np.array(labels)

In [21]:
import os

for label_name, path in data_paths.items():
    print(f"Checking files in: {path}")
    print(os.listdir(path))

Checking files in: C:/Users/Admin/Desktop/모션저장/등/좌표
['keypoints_1.npy', 'keypoints_10.npy', 'keypoints_2.npy', 'keypoints_3.npy', 'keypoints_4.npy', 'keypoints_5.npy', 'keypoints_6.npy', 'keypoints_7.npy', 'keypoints_8.npy', 'keypoints_9.npy']
Checking files in: C:/Users/Admin/Desktop/모션저장/스쿼트/좌표
['keypoints_1.npy', 'keypoints_10.npy', 'keypoints_2.npy', 'keypoints_3.npy', 'keypoints_4.npy', 'keypoints_5.npy', 'keypoints_6.npy', 'keypoints_7.npy', 'keypoints_8.npy', 'keypoints_9.npy']
Checking files in: C:/Users/Admin/Desktop/모션저장/옆구리/좌표
['keypoints_1.npy', 'keypoints_10.npy', 'keypoints_2.npy', 'keypoints_3.npy', 'keypoints_4.npy', 'keypoints_5.npy', 'keypoints_6.npy', 'keypoints_7.npy', 'keypoints_8.npy', 'keypoints_9.npy']


In [22]:
# 데이터 로드
X, y = load_data(data_paths, labels_map)

Checking files in: C:/Users/Admin/Desktop/모션저장/등/좌표
Checking files in: C:/Users/Admin/Desktop/모션저장/스쿼트/좌표
Checking files in: C:/Users/Admin/Desktop/모션저장/옆구리/좌표


In [23]:
# 데이터 크기 확인
print("데이터 크기:", X.shape, y.shape)  # Expected output: (30, frames, joints, coords), (30,)

데이터 크기: (30, 30, 33, 3) (30,)


In [25]:
# 정규화 함수 정의
def normalize_data(data):
    """
    데이터 정규화 (0~1 범위로 스케일링).
    Args:
        data (np.array): 로드된 데이터, Shape: (samples, frames, joints, coords).
    Returns:
        np.array: 정규화된 데이터.
    """
    data_min = data.min(axis=(1, 2, 3), keepdims=True)  # 각 샘플별 최소값
    data_max = data.max(axis=(1, 2, 3), keepdims=True)  # 각 샘플별 최대값
    normalized_data = (data - data_min) / (data_max - data_min + 1e-8)  # 0~1로 스케일링
    return normalized_data

In [26]:
# 데이터 정규화
X_normalized = normalize_data(X)

In [27]:
# 데이터 분할 (훈련/검증/테스트 세트)
X_train, X_temp, y_train, y_temp = train_test_split(X_normalized, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Train shape: {X_train.shape}, Validation shape: {X_val.shape}, Test shape: {X_test.shape}")

Train shape: (21, 30, 33, 3), Validation shape: (4, 30, 33, 3), Test shape: (5, 30, 33, 3)


In [28]:
# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)  # LSTM 출력
        out = self.fc(out[:, -1, :])  # 마지막 타임스텝의 출력만 사용
        return out

In [29]:
# 모델 초기화
input_dim = X_train.shape[2] * X_train.shape[3]  # 관절 수 * 좌표 수
hidden_dim = 128
output_dim = len(labels_map)  # 클래스 수 (등, 스쿼트, 옆구리)
model = LSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

In [30]:
# 데이터 로더 생성
train_dataset = TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train).long())
val_dataset = TensorDataset(torch.tensor(X_val).float(), torch.tensor(y_val).long())
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

In [31]:
# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [35]:
# 학습 루프
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        X_batch = X_batch.view(X_batch.size(0), -1, input_dim)  # 차원 변환 (batch, seq_len, input_dim)
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # 검증 단계
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.view(X_batch.size(0), -1, input_dim)  # 차원 변환
            y_pred = model(X_batch)
            val_loss += criterion(y_pred, y_batch).item()
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss / len(train_loader):.4f}, Validation Loss: {val_loss / len(val_loader):.4f}")

Epoch 1/20, Train Loss: 0.0648, Validation Loss: 0.0414
Epoch 2/20, Train Loss: 0.0412, Validation Loss: 0.0274
Epoch 3/20, Train Loss: 0.0279, Validation Loss: 0.0191
Epoch 4/20, Train Loss: 0.0203, Validation Loss: 0.0142
Epoch 5/20, Train Loss: 0.0179, Validation Loss: 0.0113
Epoch 6/20, Train Loss: 0.0118, Validation Loss: 0.0091
Epoch 7/20, Train Loss: 0.0115, Validation Loss: 0.0078
Epoch 8/20, Train Loss: 0.0099, Validation Loss: 0.0068
Epoch 9/20, Train Loss: 0.0073, Validation Loss: 0.0060
Epoch 10/20, Train Loss: 0.0074, Validation Loss: 0.0054
Epoch 11/20, Train Loss: 0.0067, Validation Loss: 0.0049
Epoch 12/20, Train Loss: 0.0054, Validation Loss: 0.0045
Epoch 13/20, Train Loss: 0.0050, Validation Loss: 0.0042
Epoch 14/20, Train Loss: 0.0045, Validation Loss: 0.0040
Epoch 15/20, Train Loss: 0.0043, Validation Loss: 0.0037
Epoch 16/20, Train Loss: 0.0040, Validation Loss: 0.0035
Epoch 17/20, Train Loss: 0.0039, Validation Loss: 0.0033
Epoch 18/20, Train Loss: 0.0036, Validat

In [36]:
from sklearn.metrics import classification_report

In [37]:
# 테스트 데이터 평가
model.eval()
y_true, y_pred = [], []
test_loader = DataLoader(TensorDataset(torch.tensor(X_test).float(), torch.tensor(y_test).long()), batch_size=4)
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.view(X_batch.size(0), -1, input_dim)  # 차원 변환
        outputs = model(X_batch)
        _, preds = torch.max(outputs, 1)
        y_true.extend(y_batch.numpy())
        y_pred.extend(preds.numpy())

print(classification_report(y_true, y_pred, target_names=list(labels_map.keys())))

              precision    recall  f1-score   support

           등       1.00      1.00      1.00         1
         스쿼트       1.00      1.00      1.00         2
         옆구리       1.00      1.00      1.00         2

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5

